In this blog post, I will continue using the Seaborn "Car Crashes" dataset. Previously, we analyzed the states in this dataset by using the k-means clustering algorithm. In this blog post, we will continue with this analysis, but instead use the DBSCAN algorithm. DBSCAN will allow us to identify states which are outliers/anomalies. 

# Get and Examine the Data

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

sns.get_dataset_names()
crashes = sns.load_dataset('car_crashes')
crashes.head()

In the clustering blog post, we already established that this dataset has no NaN values and that it is ok to jump directly into analytics/visualizations. 

Let's perform the same PCA steps that we did in the clustering blog post. 

In [ ]:
from sklearn.decomposition import PCA
import pandas as pd

pca = PCA(n_components=2)

# Remove the abbreviation column, as it is not helpful in comparing data points. 
crashes_without_abbrev = crashes.set_index('abbrev')
crashes_without_abbrev.loc[:, crashes_without_abbrev.columns != 'abbrev']
crashes_without_abbrev

crashes2D = pca.fit_transform(crashes_without_abbrev)
crashes2D = pd.DataFrame(crashes2D, index=crashes_without_abbrev.index)
pca.explained_variance_ratio_

From the explained variance ratio, we can see that nearly all of the variance (~98.7%) in the dataset is explained by the first principal component 

In [ ]:
ax = crashes2D.plot.scatter(x=0, y=1, figsize=(10,7))
for ind,row in crashes2D.iterrows():
    ax.text(row[0], row[1], ind)
ax.set_xlabel('PCA Component 1 (98.67%)');
ax.set_ylabel('PCA Component 2 (1.16%)');

Above is a plot which can help us better visualize where states lie along the two principal component axes. 

First, let's look for potential outliers by computing the Z-scores on our dataframe. Any states with z-score values that are larger than 2 or smaller than -2 are potential outliers.

In [ ]:
norm_crashes = (crashes_without_abbrev - crashes_without_abbrev.mean()) / crashes_without_abbrev.std()
norm_crashes.head() # As we can see, the data has been normalized. 

In [ ]:
cols = norm_crashes.columns.values.tolist()

norm_outliers = set()

for ind,row in norm_crashes.iterrows():
    for j in range(len(cols)):
        if(abs(row[cols[j]]) > 2):
            norm_outliers.add(ind)
norm_outliers

We get the following list of states that are potential outliers. As we can see from the PCA plot, some of these states/districts, like DC, LA, and MT, lie on the outskirts of the plot and aren't close to other states.  

However, picking an arbitrary z-score threshold to detect outliers is a somewhat crude method. Let's now use the DBSCAN algorithm to identify outliers. 

## DBSCAN Algorithm

Now, we are ready to use the DBScan algorithm to identify the outlier states. This algorithm takes in two parameters: eps and min_samples. This algorithm finds core points and then expands outwards from those core points to form clusters.

In [ ]:
from sklearn.cluster import DBSCAN
dbscan = DBSCAN(eps=30, min_samples=2)
dbscan.fit(crashes_without_abbrev)

In [ ]:
dbscan.labels_

We had to make our eps quite large and the min. samples small in order to get meaningful clusters. Otherwise, there were just too many outliers. 

Let's now plot these cluster assignments on our original PCA plot. 

In [ ]:
ax = crashes2D.plot.scatter(x=0, y=1, figsize=(10,7), c=dbscan.labels_, cmap='tab10', s=50)
for ind,row in crashes2D.iterrows():
    ax.text(row[0], row[1], ind)
ax.set_xlabel('PCA Component 1 (98.67%)');
ax.set_ylabel('PCA Component 2 (1.16%)');
# ax.axvline(x=85, color='r')

Let's try to get a list of the outliers identified by the DBSCAN algorithm. 

In [ ]:
outliers = [crashes2D.index[i] for i in range(len(dbscan.labels_)) if dbscan.labels_[i] == -1]
outliers

The DBSCAN algorithm, with eps of 30 and min_samples of 2, identifies 6 states/districts above as outliers ('CT', 'DC', 'LA', etc.). 

It seems that from the plot, some states, like LA and MD, are so far from the rest of the states that they are clear outliers. Additionally, even though 'CT' might appear close to the red cluster, remember that the PCA plot is a 2-D representation of high-dimensional data. Therefore, 'CT' is actually probably much further away from the red cluster in reality. 

## Gaussian Mixtures Algorithm

The Gaussian Mixture algorithm assumes that the data is a combination of several different normal (Gaussian) distributions. A given data point will be added to a cluster if it has the highest probability of belonging to that cluster as opposed to other clusters.  

Let's try running the Gaussian Mixture Algorithm in order to determine the anomalies in the crashes dataset. We can tune the value of num_components (i.e. number of clusters). Also, let's set n_inits to be 10. 

In [ ]:
from sklearn.mixture import GaussianMixture
gm = GaussianMixture(n_components=2, n_init=10, random_state=42)
gm.fit(crashes_without_abbrev)

Let's take a look at the parameters that were obtained from the training process. 

In [ ]:
gm.weights_

In [ ]:
gm.means_

In [ ]:
gm.converged_

In [ ]:
gm.n_iter_

It looks like the algorithm converged after 21 iterations. Now, let's take a look at the predicted class labels as well as the probability distribution between the two classes. 

In [ ]:
gm_labels = gm.predict(crashes_without_abbrev)
gm_labels

In [ ]:
gm.predict_proba(crashes_without_abbrev).round(3)[:5]

As we can see, there are two classes, with labels 0 and 1. The probability that each of the first 5 samples belongs to class 0 and class 1, respectively, are shown in the cell above.  

Let's now try to visualize the cluster assignments given by the Gaussian Mixture on the PCA plot. 

In [ ]:
ax = crashes2D.plot.scatter(x=0, y=1, figsize=(10,7), c=gm_labels, cmap='viridis', s=50)
for ind,row in crashes2D.iterrows():
    ax.text(row[0], row[1], ind)
ax.set_xlabel('PCA Component 1 (98.67%)');
ax.set_ylabel('PCA Component 2 (1.16%)');

Unsurprisingly, the samples are again split primarily based off of Principal Component 1. 

## Gaussian Mixtures for Anomaly Detection

We can use the trained Gaussian Mixture model in order to predict which observations are outliers. Visually, it appears that there should be at least 2 or 3 outliers. As a first attempt, we can set a density threshold at the 5th percentile. This would account for approximately 2-3 samples (0.05 * 51 = 2.55). All of the samples that fall below this threshold will be classified as outliers. 

In [ ]:
import numpy as np

densities = gm.score_samples(crashes_without_abbrev)
density_threshold = np.percentile(densities, 5)
anomalies_below = crashes_without_abbrev[densities < density_threshold]
anomalies_below

When using Gaussian Mixtures, Kentucky, Mississippi, and North Dakota are classified as outliers. This is an interesting result, since DBSCAN identified a different set of outliers.  

One possible explanation, is that DBSCAN has difficulty with sparse points, while Gaussian Mixtures is more robust to sparse points. Gaussian Mixtures instead focuses on the probability that a sample belongs to a given cluster or not. Hence, the DBSCAN and Gaussian Mixtures algorithms may identify a different set of outliers in some cases. 

# Attributions

1. https://www.w3schools.com/python/pandas/ref_df_iterrows.asp
2. https://en.wikipedia.org/wiki/DBSCAN
3. https://sparkbyexamples.com/pandas/pandas-get-column-names/
4. https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html
5. https://scikit-learn.org/stable/modules/generated/sklearn.mixture.GaussianMixture.html#sklearn.mixture.GaussianMixture
6. https://www.geeksforgeeks.org/gaussian-mixture-model/
